In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly 
import plotly.graph_objects as go
import plotly.express as px

from sklearn import linear_model
%matplotlib qt
pd.set_option('mode.chained_assignment', None)

# Load raw and save ready data

In [ ]:
# path = f'Experiments\MultiplyTemperature\Exp5(12.5)\Exp5_down.csv'

path = ''
while (path == '') or (not os.path.isfile(path)):
    path = input(f"Input data path: ")
_path_list = (path).split('\\')
folder_path = '\\'.join(_path_list[:-1])
data_name = _path_list[-1]
print(f'    Folder:\n{folder_path}\n    Data name:\n {data_name}')

In [ ]:
path

In [ ]:
df1 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up1.csv',
                  index_col=0)
# df2 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up2.csv',index_col=0)
# df3 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up3.csv',index_col=0)
# df4 = pd.read_csv('Experiments\\MultiplyTemperature\\Exp0(0)\\Exp0_up4.csv',index_col=0)


In [ ]:
df1

In [ ]:
rules = {
    1: 'image_sweep_check',
    2: 'combine_check',  # 'inner_processor_check':'OK_inner'
    0: 'OK'
}
col_rules = {
    'Viscosity_mark': 'Viscosity_verbose',
    'Temperature_mark': 'Temperature_verbose',
}

df1 = df1.rename(columns=col_rules)
df1[['Viscosity_verbose','Temperature_verbose']] =\
    df1[['Viscosity_verbose','Temperature_verbose']].replace(rules)

# df2 = df2.rename(columns=col_rules)
# df2[['Viscosity_verbose','Temperature_verbose']] = df2[['Viscosity_verbose','Temperature_verbose']].replace(rules)
# df2['time'] =df2['time']+ df1['time'].max()+1

# df3 = df3.rename(columns=col_rules)
# df3[['Viscosity_verbose','Temperature_verbose']] = df3[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df3['time'] =df3['time']+ df2['time'].max()+1

# df4 = df4.rename(columns=col_rules)
# df4[['Viscosity_verbose','Temperature_verbose']] = df4[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df4['time'] =df4['time']+ df3['time'].max()+1
# df = pd.concat([df1,df2,df3,df4])

df = df1.dropna(subset=['Viscosity', 'Temperature'])
df

In [ ]:
np.any(df['time'].duplicated())

In [ ]:
assert not os.path.isfile(f'{folder_path}\AData_{data_name}'), 'Data already exists'
df.to_csv(f'{folder_path}\AData_{data_name}', index=0)


# Load

## Load df to compare

In [11]:
path2 = ''
while (path2 == '') or (not os.path.isfile(path2)):
    path2 = input(f"Input data path: ")
_path_list2 = (path).split('\\')
folder_path2 = '\\'.join(_path_list2[:-1])
data_name2 = _path_list2[-1]
df2 = pd.read_csv(path2)
print(f'Loaded: {data_name2}')
df2

Loaded: AData_Exp2_down.csv


,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
0,5.0,159.6,image_sweep_check,14.4,OK
1,6.0,159.6,OK_inner,14.4,OK
2,7.0,159.3,OK,14.4,OK
3,8.0,159.0,OK_inner,14.4,OK
4,9.0,159.0,image_sweep_check,14.4,OK
...,...,...,...,...,...
2828,3318.0,31.7,image_sweep_check,39.9,OK
2829,3319.0,31.6,OK,39.9,OK
2830,3320.0,31.7,image_sweep_check,39.9,OK
2831,3321.0,31.7,image_sweep_check,39.9,OK


## Load base

In [2]:
df2 = pd.DataFrame()
path = ''
while (path == '') or (not os.path.isfile(path)):
    path = input(f"Input data path: ")
_path_list = (path).split('\\')
folder_path = '\\'.join(_path_list[:-1])
data_name = _path_list[-1]
df = pd.read_csv(path)
print(f'Loaded: {data_name}')
df

,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
0,0.0,38.2,OK,38.1,image_sweep_check
1,1.0,38.2,OK,38.1,OK
2,2.0,38.2,OK,38.1,OK
3,3.0,38.1,image_sweep_check,38.2,OK
4,4.0,38.0,OK_inner,38.2,OK
...,...,...,...,...,...
7530,7674.0,155.4,image_sweep_check,14.9,OK
7531,7675.0,159.3,OK,14.9,OK
7532,7676.0,159.3,image_sweep_check,14.9,OK
7533,7677.0,157.8,OK_inner,14.9,OK


# Process

In [14]:
regression_done = False
temperature_cond = ((12 < df['Temperature']) & (df['Temperature'] < 45))
viscosity_cond = ((0 < df['Viscosity']) & (df['Viscosity'] < 200))

dfs = df[temperature_cond & viscosity_cond]

temperature_cond2 = ((12 < df2['Temperature']) & (df2['Temperature'] < 45))
viscosity_cond2 = ((0 < df2['Viscosity']) & (df2['Viscosity'] < 200))
dfs2 = df2[temperature_cond2 & viscosity_cond2]
# dfs =df

In [ ]:
# plot_rules = {'processor_sweep':2, 'OK':0, 'error':4, 'value_combine':3,
#        'inner_processor_check':1}
# df[['Viscosity_verbose','Temperature_verbose']] = df[['Viscosity_verbose','Temperature_verbose']].replace(plot_rules)
# dfs['time'] = dfs['time'] / 60


In [ ]:
# T_goupT= dfs.groupby(by='Temperature')['Temperature']
# T_groupV=dfs.groupby(by='Temperature')['Viscosity']
# def foo(g):
#     whisker_width=0
#     q1 = g.quantile(0.45)
#     q3 = g.quantile(0.55)
#     iqr = q3 - q1
#     return (g >= q1 - whisker_width*iqr) & (g <= q3 + whisker_width*iqr)
#     # return np.abs((g - g.median()) / iqr) < 2.22
#     # return np.abs((g - g.mean()) / g.std(ddof=0)) < 1

# error_mask= T_groupV.apply(foo ).droplevel([0]).reset_index(drop=True).to_numpy()

# dfs = dfs[error_mask]

# # T_goupT2= dfs2.groupby(by='Temperature')['Temperature']
# # T_groupV2=dfs2.groupby(by='Temperature')['Viscosity']
# # dfs2
# dfs

In [20]:
dfs_median = dfs[['Temperature',
                  'Viscosity']].groupby(by='Temperature').median()
dfs_mean = dfs[['Temperature', 'Viscosity']].groupby(by='Temperature').mean()

dfs_median2 = dfs2[['Temperature',
                    'Viscosity']].groupby(by='Temperature').median()
dfs_mean2 = dfs2[['Temperature', 'Viscosity']].groupby(by='Temperature').mean()


## Time plots

In [18]:
fig, ax_v = plt.subplots()
ax_T = ax_v.twinx()

ax_v.scatter(dfs['time'], dfs['Viscosity'], color='red', marker='.')
ax_v.set_ylabel('Viscosity', color='red')

ax_T.scatter(dfs['time'], dfs['Temperature'], color='blue', marker='.')
ax_T.set_ylabel('Temperature', color='blue')


In [ ]:
dfs['Temperature_verbose'].unique()

In [ ]:
colors_dict = {
    'OK': '#636efa',
    'OK_inner': '#b9bdf9',
    'image_sweep_check': '#EF553B',
    'combine_check': '#00cc96'
}
# '#EF553B' - red
fig = go.Figure()
fig.add_traces([
    go.Scatter(
        x=dfs['time'],
        y=dfs['Viscosity'],
        name='Viscosity',
        mode='markers',
        marker=dict(size=5,
                    color=dfs['Viscosity_verbose'].replace(colors_dict))),
    go.Scatter(
        x=dfs['time'],
        y=dfs['Temperature'],
        name='Temperature',
        yaxis='y2',
        mode='markers',
        marker=dict(size=5,
                    symbol='square',
                    color=dfs['Temperature_verbose'].replace(colors_dict)),
    ),
])

fig.update_layout(
    hovermode="x unified",
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    yaxis2=dict(overlaying='y', side='right'),
)
fig.show()

## Temterature plots

In [30]:
fig = go.Figure()
fig = px.scatter(
    dfs,
    x="Temperature",
    y="Viscosity",
    color='Viscosity_verbose',
)

fig.add_traces([
    go.Scatter(name='Median',
               x=dfs_median.index,
               y=dfs_median['Viscosity'],
               mode='lines',
               line=dict(color='black')),
    go.Scatter(name='Mean',
               x=dfs_mean.index,
               y=dfs_mean['Viscosity'],
               mode='lines',
               line=dict(color='red')),
])
if len(dfs2):
    fig.add_traces([
        go.Scatter(name='Median2',
                   x=dfs_median.index,
                   y=dfs_median2['Viscosity'],
                   mode='lines',
                   line=dict(color='black', width=2, dash='dot')),
        go.Scatter(name='Mean2',
                   x=dfs_mean2.index,
                   y=dfs_mean2['Viscosity'],
                   mode='lines',
                   line=dict(color='red', width=2, dash='dot')),
    ])

if regression_done:
    x = np.array(dfs_median.index)
    y = nu0 * np.exp(EkJ * 1000 / 8.314 / (x+273.15))
    fig.add_trace(go.Scatter(name='Regression', x=x, y=y, mode='lines'))
fig.update_layout(hovermode="x unified", margin=dict(l=0, r=0, b=0, t=0, pad=0))
fig.show()

## OLS

In [ ]:
dfp = dfs_median
dfp

In [ ]:
T = np.array(dfp.index)
nu = np.array(dfp['Viscosity'])

In [ ]:
y_lnD = np.log(nu)
x_T = 1 / (T+273.15)

X = np.array([x_T]).T
Y = y_lnD

In [ ]:
reg = linear_model.LinearRegression(fit_intercept=True)
reg.fit(X, Y)
w = reg.coef_[0]
w0 = reg.intercept_

In [ ]:
nu0 = np.exp(w0)
EkJ = 8.314 * w / 1000


In [ ]:
exp_name = folder_path.split('(')[-1][:-1]
print(
    f"Experiment= {data_name[6:-4]}: {exp_name}%",
    f'nu0=  {nu0: 6.3e}',
    f'E  =  {EkJ: 6.3f} kJ',
    sep='\n',
)
regression_done = True


## Saving regression

In [ ]:
dfp = dfs_mean

In [ ]:
assert not os.path.isfile(f'{folder_path}\BReg_{data_name[6:]}'), 'Data already exists'
dfp.to_csv(f'{folder_path}\BReg_{data_name[6:]}', index=0)